In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, RepeatedKFold
from xgboost import XGBRegressor

In [15]:
data = pd.read_csv('synthetic_data_gaussian_copula.csv')
data = data.sort_values(by='Date/Time')
data.shape

(1000, 37)

Prediction of Arsenic

In [ ]:
target = 'As [ng/m³]' 
features = [col for col in data.columns if col not in ['No', 'Date/Time', 'Date/time end', 'Altitude [m]', target]]
X = data[features]
y = data[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'No', 'Date/Time', 'Date/time end',
       'Altitude [m]', 'Size fraction', 'Mass v [µg/m**3]', 'Na+ [µg/m**3]',
       '[NH4]+ [µg/m**3]', 'K+ [µg/m**3]', 'Mg2+ [µg/m**3]', 'Ca2+ [µg/m**3]',
       'Cl- [µg/m**3]', '[NO3]- [µg/m**3]', '[SO4]2- [µg/m**3]',
       '[C2O4]2- [µg/m**3]', 'Br- [µg/m**3]', 'C org [µg/m**3]',
       'EC [µg/m**3]', 'TC [µg/m**3]', 'Ca [ng/m**3]', 'Ti [ng/m**3]',
       'V [ng/m**3]', 'Cr [ng/m**3]', 'Mn [ng/m**3]', 'Fe [ng/m**3]',
       'Ni [ng/m**3]', 'Cu [ng/m**3]', 'Zn [ng/m**3]', 'Pb [ng/m**3]',
       'As [ng/m**3]', 'Se [ng/m**3]', 'Sr [ng/m**3]', 'Rb [ng/m**3]',
       'Ba [ng/m**3]', 'La [ng/m**3]', 'Ce [ng/m**3]'],
      dtype='object')

Gradient Boosted Tree(Xgboost)

In [ ]:
# Define and train the Gradient Boosting model
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Regression task with squared error loss
    n_estimators=100,             # Number of boosting rounds (trees)
    learning_rate=0.1,            # Step size shrinkage to prevent overfitting
    max_depth=3,                  # Maximum depth of each tree
    random_state=42               # For reproducibility
)

model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val)

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
df = pd.read_csv('synthetic_data_gaussian_copula.csv')

# Define columns
targets = ['Zn [ng/m**3]', 'As [ng/m**3]']
drop_cols = ['No', 'Date/Time', 'Date/time end', 'Duration', 'Altitude [m]']
categorical_features = ['Size fraction']
numerical_features = [col for col in df.columns if col not in targets + drop_cols + categorical_features]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))
])

# Hyperparameter grid
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 4, 5,6],
    'regressor__learning_rate': [0.05, 0.1, 0.2,0.3],
}

# K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_mean_absolute_error', refit=True)
grid_search.fit(df.drop(columns=targets + drop_cols), df[targets])

# Best model
best_model = grid_search.best_estimator_

# Cross-validated predictions
y_pred = cross_val_predict(best_model, df.drop(columns=targets + drop_cols), df[targets], cv=kf)

# Evaluate
zn_true = df['Zn [ng/m**3]']
as_true = df['As [ng/m**3]']
zn_pred = y_pred[:, 0]  # Zn predictions
as_pred = y_pred[:, 1]  # As predictions

print(f"The best parameters for the model are: {grid_search.best_params_}")
print("Evaluation for Zn [ng/m**3]:")
print(f"MAE: {mean_absolute_error(zn_true, zn_pred)}")
print(f"RMSE: {np.sqrt(mean_squared_error(zn_true, zn_pred))}")
print(f"MSE: {mean_squared_error(zn_true, zn_pred)}")
print(f"R2: {r2_score(zn_true, zn_pred)}")

print("\nEvaluation for As [ng/m**3]:")
print(f"MAE: {mean_absolute_error(as_true, as_pred)}")
print(f"RMSE: {np.sqrt(mean_squared_error(as_true, as_pred))}")
print(f"MSE: {mean_squared_error(as_true, as_pred)}")
print(f"R2: {r2_score(as_true, as_pred)}")

The best parameters for the model are: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 3, 'regressor__n_estimators': 200}
Evaluation for Zn [ng/m**3]:
MAE: 0.4036473855851516
RMSE: 0.6515366125284167
MSE: 0.4244999574650041
R2: 0.6654635291387763

Evaluation for As [ng/m**3]:
MAE: 0.07152986233579167
RMSE: 0.12683739358513269
MSE: 0.01608772441146986
R2: -0.2179146881610341


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

# Define evaluation metrics from the paper
def nse(observed, predicted):
    """Nash-Sutcliffe Efficiency"""
    numerator = np.sum((observed - predicted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    return 1 - (numerator / denominator)

def pcc(observed, predicted):
    """Pearson Correlation Coefficient"""
    return np.corrcoef(observed, predicted)[0, 1]

def mape(observed, predicted):
    """Mean Absolute Percentage Error"""
    observed = np.where(observed == 0, 1e-10, observed)
    return np.mean(np.abs((observed - predicted) / observed)) * 100

def pbias(observed, predicted):
    """Percent Bias"""
    return np.sum(predicted - observed) / np.sum(observed) * 100

# Custom NSE scorer for multi-output
def nse_scorer(y_true, y_pred):
    numerator = np.sum((y_true - y_pred) ** 2, axis=0)
    denominator = np.sum((y_true - np.mean(y_true, axis=0)) ** 2, axis=0)
    nse = 1 - numerator / denominator
    return np.mean(nse)  # Average across outputs

# Load the dataset
df = pd.read_csv('synthetic_data_gaussian_copula.csv')

# Define columns
targets = ['Zn [ng/m³]', 'As [ng/m³]']
drop_cols = ['No', 'Date/Time', 'Date/time end', 'Duration', 'Altitude [m]']
categorical_features = ['Size fraction']
numerical_features = [col for col in df.columns if col not in targets + drop_cols + categorical_features]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Model pipeline with MultiOutputRegressor for AdaBoost
base_estimator = DecisionTreeRegressor(random_state=42)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(AdaBoostRegressor(base_estimator=base_estimator, random_state=42)))
])
# Hyperparameter grid
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 4, 5,6],
    'regressor__learning_rate': [0.05, 0.1, 0.2,0.3],
}

# K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_mean_absolute_error', refit=True)
grid_search.fit(df.drop(columns=targets + drop_cols), df[targets])

# Best model
best_model = grid_search.best_estimator_

# Cross-validated predictions
y_pred = cross_val_predict(best_model, df.drop(columns=targets + drop_cols), df[targets], cv=kf)

# Evaluate
zn_true = df['Zn [ng/m**3]']
as_true = df['As [ng/m**3]']
zn_pred = y_pred[:, 0]  # Zn predictions
as_pred = y_pred[:, 1]  # As predictions

print(f"The best parameters for the model are: {grid_search.best_params_}")
print("Evaluation for Zn [ng/m**3]:")
print(f"MAE: {mean_absolute_error(zn_true, zn_pred)}")
print(f"RMSE: {np.sqrt(mean_squared_error(zn_true, zn_pred))}")
print(f"MSE: {mean_squared_error(zn_true, zn_pred)}")
print(f"R2: {r2_score(zn_true, zn_pred)}")

print("\nEvaluation for As [ng/m**3]:")
print(f"MAE: {mean_absolute_error(as_true, as_pred)}")
print(f"RMSE: {np.sqrt(mean_squared_error(as_true, as_pred))}")
print(f"MSE: {mean_squared_error(as_true, as_pred)}")
print(f"R2: {r2_score(as_true, as_pred)}")

TypeError: AdaBoostRegressor.__init__() got an unexpected keyword argument 'base_estimator'

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

# Define evaluation metrics from the paper
def nse(observed, predicted):
    """Nash-Sutcliffe Efficiency"""
    numerator = np.sum((observed - predicted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    return 1 - (numerator / denominator)

def pcc(observed, predicted):
    """Pearson Correlation Coefficient"""
    return np.corrcoef(observed, predicted)[0, 1]

def mape(observed, predicted):
    """Mean Absolute Percentage Error"""
    observed = np.where(observed == 0, 1e-10, observed)
    return np.mean(np.abs((observed - predicted) / observed)) * 100

def pbias(observed, predicted):
    """Percent Bias"""
    return np.sum(predicted - observed) / np.sum(observed) * 100

# Custom NSE scorer for multi-output
def nse_scorer(y_true, y_pred):
    numerator = np.sum((y_true - y_pred) ** 2, axis=0)
    denominator = np.sum((y_true - np.mean(y_true, axis=0)) ** 2, axis=0)
    nse = 1 - numerator / denominator
    return np.mean(nse)  # Average across outputs

# Load the dataset
df = pd.read_csv('synthetic_data_gaussian_copula.csv')

# Define columns
targets = ['Zn [ng/m**3]', 'As [ng/m**3]']
drop_cols = ['No', 'Date/Time', 'Date/time end', 'Duration', 'Altitude [m]']
categorical_features = ['Size fraction']
numerical_features = [col for col in df.columns if col not in targets + drop_cols + categorical_features]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Model pipeline with MultiOutputRegressor for AdaBoost
decision_model = DecisionTreeRegressor(random_state=42)  # Removed fixed max_depth
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(AdaBoostRegressor(estimator=decision_model, random_state=42)))
])

# Hyperparameter grid
param_grid = {
    'regressor__estimator__n_estimators': [50, 100, 200],
    'regressor__estimator__learning_rate': [0.01, 0.1, 1.0],
    'regressor__estimator__estimator__max_depth': [3, 4, 5, 6]  # Corrected to target DecisionTreeRegressor
}

# Sort data by time for consistency (though only KFold used here)
df['Date/Time'] = pd.to_datetime(df['Date/Time'])
df = df.sort_values('Date/Time')

# Prepare features and targets
X = df.drop(columns=targets + drop_cols)
y = df[targets]

# Define scoring metrics
scoring = {
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'r2': 'r2',
    'nse': make_scorer(nse_scorer, greater_is_better=True)
}

# Standard k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
grid_search_kf = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit='neg_mean_absolute_error')
grid_search_kf.fit(X, y)

# Best model from k-fold
best_model_kf = grid_search_kf.best_estimator_

# Cross-validated predictions for k-fold
y_pred_kf = cross_val_predict(best_model_kf, X, y, cv=kf)

# Evaluate for k-fold
zn_true = df['Zn [ng/m**3]']
as_true = df['As [ng/m**3]']
zn_pred_kf = y_pred_kf[:, 0]  # Zn predictions
as_pred_kf = y_pred_kf[:, 1]  # As predictions

print("Standard k-Fold Cross-Validation (k=10)")
print(f"Best parameters: {grid_search_kf.best_params_}")
print(f"Best scores: {grid_search_kf.best_score_:.4f} (neg_mean_absolute_error)")
print(f"CV results for other metrics:")
for metric in scoring:
    mean_score = grid_search_kf.cv_results_[f'mean_test_{metric}'][grid_search_kf.best_index_]
    std_score = grid_search_kf.cv_results_[f'std_test_{metric}'][grid_search_kf.best_index_]
    print(f"{metric}: {mean_score:.4f} ± {std_score:.4f}")

print("\nEvaluation for Zn [ng/m³]:")
print(f"MAE: {mean_absolute_error(zn_true, zn_pred_kf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(zn_true, zn_pred_kf)):.4f}")
print(f"MSE: {mean_squared_error(zn_true, zn_pred_kf):.4f}")
print(f"R2: {r2_score(zn_true, zn_pred_kf):.4f}")
print(f"NSE: {nse(zn_true, zn_pred_kf):.4f}")
print(f"PCC: {pcc(zn_true, zn_pred_kf):.4f}")
print(f"MAPE: {mape(zn_true, zn_pred_kf):.4f}%")
print(f"PBIAS: {pbias(zn_true, zn_pred_kf):.4f}%")

print("\nEvaluation for As [ng/m³]:")
print(f"MAE: {mean_absolute_error(as_true, as_pred_kf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(as_true, as_pred_kf)):.4f}")
print(f"MSE: {mean_squared_error(as_true, as_pred_kf):.4f}")
print(f"R2: {r2_score(as_true, as_pred_kf):.4f}")
print(f"NSE: {nse(as_true, as_pred_kf):.4f}")
print(f"PCC: {pcc(as_true, as_pred_kf):.4f}")
print(f"MAPE: {mape(as_true, as_pred_kf):.4f}%")
print(f"PBIAS: {pbias(as_true, as_pred_kf):.4f}%")

Standard k-Fold Cross-Validation (k=10)
Best parameters: {'regressor__estimator__estimator__max_depth': 6, 'regressor__estimator__learning_rate': 0.01, 'regressor__estimator__n_estimators': 50}
Best scores: -0.2337 (neg_mean_absolute_error)
CV results for other metrics:
neg_mean_absolute_error: -0.2337 ± 0.0255
neg_mean_squared_error: -0.2059 ± 0.0595
r2: 0.3279 ± 0.0435
nse: 0.3279 ± 0.0435

Evaluation for Zn [ng/m³]:
MAE: 0.4102
RMSE: 0.6313
MSE: 0.3985
R2: 0.6860
NSE: 0.6860
PCC: 0.8314
MAPE: 77.1489%
PBIAS: -5.7323%

Evaluation for As [ng/m³]:
MAE: 0.0573
RMSE: 0.1154
MSE: 0.0133
R2: -0.0079
NSE: -0.0079
PCC: 0.1010
MAPE: 793.6263%
PBIAS: -27.9140%


Relevance Vector Regressor

In [33]:
"""Relevance Vector Machine classes for regression and classification."""
import numpy as np

from scipy.optimize import minimize
from scipy.special import expit

from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.metrics.pairwise import (
    linear_kernel,
    rbf_kernel,
    polynomial_kernel
)
from sklearn.multiclass import OneVsOneClassifier
from sklearn.utils.validation import check_X_y


class BaseRVM(BaseEstimator):

    """Base Relevance Vector Machine class.

    Implementation of Mike Tipping's Relevance Vector Machine using the
    scikit-learn API. Add a posterior over weights method and a predict
    in subclass to use for classification or regression.
    """

    def __init__(
        self,
        kernel='rbf',
        degree=3,
        coef1=None,
        coef0=0.0,
        n_iter=3000,
        tol=1e-3,
        alpha=1e-6,
        threshold_alpha=1e9,
        beta=1.e-6,
        beta_fixed=False,
        bias_used=True,
        verbose=False
    ):
        """Copy params to object properties, no validation."""
        self.kernel = kernel
        self.degree = degree
        self.coef1 = coef1
        self.coef0 = coef0
        self.n_iter = n_iter
        self.tol = tol
        self.alpha = alpha
        self.threshold_alpha = threshold_alpha
        self.beta = beta
        self.beta_fixed = beta_fixed
        self.bias_used = bias_used
        self.verbose = verbose

    def get_params(self, deep=True):
        """Return parameters as a dictionary."""
        params = {
            'kernel': self.kernel,
            'degree': self.degree,
            'coef1': self.coef1,
            'coef0': self.coef0,
            'n_iter': self.n_iter,
            'tol': self.tol,
            'alpha': self.alpha,
            'threshold_alpha': self.threshold_alpha,
            'beta': self.beta,
            'beta_fixed': self.beta_fixed,
            'bias_used': self.bias_used,
            'verbose': self.verbose
        }
        return params

    def set_params(self, **parameters):
        """Set parameters using kwargs."""
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def _apply_kernel(self, x, y):
        """Apply the selected kernel function to the data."""
        if self.kernel == 'linear':
            phi = linear_kernel(x, y)
        elif self.kernel == 'rbf':
            phi = rbf_kernel(x, y, self.coef1)
        elif self.kernel == 'poly':
            phi = polynomial_kernel(x, y, self.degree, self.coef1, self.coef0)
        elif callable(self.kernel):
            phi = self.kernel(x, y)
            if len(phi.shape) != 2:
                raise ValueError(
                    "Custom kernel function did not return 2D matrix"
                )
            if phi.shape[0] != x.shape[0]:
                raise ValueError(
                    "Custom kernel function did not return matrix with rows"
                    " equal to number of data points."""
                )
        else:
            raise ValueError("Kernel selection is invalid.")

        if self.bias_used:
            phi = np.append(phi, np.ones((phi.shape[0], 1)), axis=1)

        return phi

    def _prune(self):
        """Remove basis functions based on alpha values."""
        keep_alpha = self.alpha_ < self.threshold_alpha

        if not np.any(keep_alpha):
            keep_alpha[0] = True
            if self.bias_used:
                keep_alpha[-1] = True

        if self.bias_used:
            if not keep_alpha[-1]:
                self.bias_used = False
            self.relevance_ = self.relevance_[keep_alpha[:-1]]
        else:
            self.relevance_ = self.relevance_[keep_alpha]

        self.alpha_ = self.alpha_[keep_alpha]
        self.alpha_old = self.alpha_old[keep_alpha]
        self.gamma = self.gamma[keep_alpha]
        self.phi = self.phi[:, keep_alpha]
        self.sigma_ = self.sigma_[np.ix_(keep_alpha, keep_alpha)]
        self.m_ = self.m_[keep_alpha]

    def fit(self, X, y):
        """Fit the RVR to the training data."""
        X, y = check_X_y(X, y)

        n_samples, n_features = X.shape

        self.phi = self._apply_kernel(X, X)

        n_basis_functions = self.phi.shape[1]

        self.relevance_ = X
        self.y = y

        self.alpha_ = self.alpha * np.ones(n_basis_functions)
        self.beta_ = self.beta

        self.m_ = np.zeros(n_basis_functions)

        self.alpha_old = self.alpha_

        for i in range(self.n_iter):
            self._posterior()

            self.gamma = 1 - self.alpha_*np.diag(self.sigma_)
            self.alpha_ = self.gamma/(self.m_ ** 2)

            if not self.beta_fixed:
                self.beta_ = (n_samples - np.sum(self.gamma))/(
                    np.sum((y - np.dot(self.phi, self.m_)) ** 2))

            self._prune()

            if self.verbose:
                print("Iteration: {}".format(i))
                print("Alpha: {}".format(self.alpha_))
                print("Beta: {}".format(self.beta_))
                print("Gamma: {}".format(self.gamma))
                print("m: {}".format(self.m_))
                print("Relevance Vectors: {}".format(self.relevance_.shape[0]))
                print()

            delta = np.amax(np.absolute(self.alpha_ - self.alpha_old))

            if delta < self.tol and i > 1:
                break

            self.alpha_old = self.alpha_

        if self.bias_used:
            self.bias = self.m_[-1]
        else:
            self.bias = None

        return self


class RVR(BaseRVM, RegressorMixin):

    """Relevance Vector Machine Regression.

    Implementation of Mike Tipping's Relevance Vector Machine for regression
    using the scikit-learn API.
    """

    def _posterior(self):
        """Compute the posterior distriubtion over weights."""
        i_s = np.diag(self.alpha_) + self.beta_ * np.dot(self.phi.T, self.phi)
        self.sigma_ = np.linalg.inv(i_s)
        self.m_ = self.beta_ * np.dot(self.sigma_, np.dot(self.phi.T, self.y))

    def predict(self, X, eval_MSE=False):
        """Evaluate the RVR model at x."""
        phi = self._apply_kernel(X, self.relevance_)

        y = np.dot(phi, self.m_)

        if eval_MSE:
            MSE = (1/self.beta_) + np.dot(phi, np.dot(self.sigma_, phi.T))
            return y, MSE[:, 0]
        else:
            return y


In [36]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.special import expit
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics.pairwise import linear_kernel, rbf_kernel, polynomial_kernel
from sklearn.utils.validation import check_X_y
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor

# Define the BaseRVM class
class BaseRVM(BaseEstimator):
    def __init__(
        self,
        kernel='rbf',
        degree=3,
        coef1=None,
        coef0=0.0,
        n_iter=3000,
        tol=1e-3,
        alpha=1e-6,
        threshold_alpha=1e9,
        beta=1e-6,
        beta_fixed=False,
        bias_used=True,
        verbose=False
    ):
        self.kernel = kernel
        self.degree = degree
        self.coef1 = coef1
        self.coef0 = coef0
        self.n_iter = n_iter
        self.tol = tol
        self.alpha = alpha
        self.threshold_alpha = threshold_alpha
        self.beta = beta
        self.beta_fixed = beta_fixed
        self.bias_used = bias_used
        self.verbose = verbose

    def get_params(self, deep=True):
        return {
            'kernel': self.kernel,
            'degree': self.degree,
            'coef1': self.coef1,
            'coef0': self.coef0,
            'n_iter': self.n_iter,
            'tol': self.tol,
            'alpha': self.alpha,
            'threshold_alpha': self.threshold_alpha,
            'beta': self.beta,
            'beta_fixed': self.beta_fixed,
            'bias_used': self.bias_used,
            'verbose': self.verbose
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def _apply_kernel(self, x, y):
        if self.kernel == 'linear':
            phi = linear_kernel(x, y)
        elif self.kernel == 'rbf':
            phi = rbf_kernel(x, y, self.coef1)
        elif self.kernel == 'poly':
            phi = polynomial_kernel(x, y, self.degree, self.coef1, self.coef0)
        elif callable(self.kernel):
            phi = self.kernel(x, y)
            if len(phi.shape) != 2:
                raise ValueError("Custom kernel function did not return 2D matrix")
            if phi.shape[0] != x.shape[0]:
                raise ValueError("Custom kernel function did not return matrix with rows equal to number of data points")
        else:
            raise ValueError("Kernel selection is invalid")
        if self.bias_used:
            phi = np.append(phi, np.ones((phi.shape[0], 1)), axis=1)
        return phi

    def _prune(self):
        keep_alpha = self.alpha_ < self.threshold_alpha
        if not np.any(keep_alpha):
            keep_alpha[0] = True
            if self.bias_used:
                keep_alpha[-1] = True
        self.relevance_ = self.relevance_[keep_alpha[:-1]] if self.bias_used else self.relevance_[keep_alpha]
        self.alpha_ = self.alpha_[keep_alpha]
        self.alpha_old = self.alpha_old[keep_alpha]
        self.gamma = self.gamma[keep_alpha]
        self.phi = self.phi[:, keep_alpha]
        self.sigma_ = self.sigma_[np.ix_(keep_alpha, keep_alpha)]
        self.m_ = self.m_[keep_alpha]

# Define the RVR class with numerical stability improvements
class RVR(BaseRVM, RegressorMixin):
    def _posterior(self):
        i_s = np.diag(self.alpha_) + self.beta_ * np.dot(self.phi.T, self.phi)
        self.sigma_ = np.linalg.pinv(i_s)  # Use pinv for numerical stability
        self.m_ = self.beta_ * np.dot(self.sigma_, np.dot(self.phi.T, self.y))

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        n_samples, n_features = X.shape
        self.phi = self._apply_kernel(X, X)
        n_basis_functions = self.phi.shape[1]
        self.relevance_ = X
        self.y = y
        self.alpha_ = self.alpha * np.ones(n_basis_functions)
        self.beta_ = self.beta
        self.m_ = np.zeros(n_basis_functions)
        self.alpha_old = self.alpha_
        for i in range(self.n_iter):
            self._posterior()
            self.gamma = 1 - self.alpha_ * np.diag(self.sigma_)
            self.alpha_ = self.gamma / (self.m_ ** 2 + 1e-10)  # Add epsilon to avoid division by zero
            if not self.beta_fixed:
                self.beta_ = (n_samples - np.sum(self.gamma)) / np.sum((y - np.dot(self.phi, self.m_)) ** 2)
            self._prune()
            if self.verbose:
                print(f"Iteration: {i}")
                print(f"Alpha: {self.alpha_}")
                print(f"Beta: {self.beta_}")
                print(f"Gamma: {self.gamma}")
                print(f"m: {self.m_}")
                print(f"Relevance Vectors: {self.relevance_.shape[0]}")
            delta = np.amax(np.absolute(self.alpha_ - self.alpha_old))
            if delta < self.tol and i > 1:
                break
            self.alpha_old = self.alpha_
        if self.bias_used:
            self.bias = self.m_[-1]
        else:
            self.bias = None
        return self

    def predict(self, X, eval_MSE=False):
        phi = self._apply_kernel(X, self.relevance_)
        y = np.dot(phi, self.m_)
        if eval_MSE:
            MSE = (1 / self.beta_) + np.dot(phi, np.dot(self.sigma_, phi.T))
            return y, MSE[:, 0]
        else:
            return y

# Define evaluation metrics
def nse(observed, predicted):
    numerator = np.sum((observed - predicted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    return 1 - (numerator / denominator)

def pcc(observed, predicted):
    return np.corrcoef(observed, predicted)[0, 1]

def mape(observed, predicted):
    observed = np.where(observed == 0, 1e-10, observed)
    return np.mean(np.abs((observed - predicted) / observed)) * 100

def pbias(observed, predicted):
    return np.sum(predicted - observed) / np.sum(observed) * 100

# Custom NSE scorer for multi-output
def nse_scorer(y_true, y_pred):
    numerator = np.sum((y_true - y_pred) ** 2, axis=0)
    denominator = np.sum((y_true - np.mean(y_true, axis=0)) ** 2, axis=0)
    nse = 1 - numerator / denominator
    return np.mean(nse)

# Load the dataset
df = pd.read_csv('synthetic_data_gaussian_copula.csv')

# Define columns
targets = ['Zn [ng/m**3]', 'As [ng/m**3]']
drop_cols = ['No', 'Date/Time', 'Date/time end', 'Duration', 'Altitude [m]']
categorical_features = ['Size fraction']
numerical_features = [col for col in df.columns if col not in targets + drop_cols + categorical_features]

# Set up the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Create the pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(RVR(n_iter=100, verbose=False)))  # Reduced n_iter for speed
])

# Define hyperparameter grid
param_grid = {
    'regressor__estimator__kernel': ['rbf', 'linear'],
    'regressor__estimator__coef1': [0.1, 1.0, 10.0],  # Only affects RBF kernel
    'regressor__estimator__alpha': [1e-6, 1e-4, 1e-2],
    'regressor__estimator__beta': [1e-6, 1e-4, 1e-2]
}

# Sort data by time for TimeSeriesSplit
df['Date/Time'] = pd.to_datetime(df['Date/Time'])
df = df.sort_values('Date/Time')

# Prepare features and targets
X = df.drop(columns=targets + drop_cols)
y = df[targets]

# Define scoring metrics
scoring = {
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'r2': 'r2',
    'nse': make_scorer(nse_scorer, greater_is_better=True)
}

# Perform k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
grid_search_kf = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit='neg_mean_absolute_error')
grid_search_kf.fit(X, y)

# Get the best model and predictions for k-fold
best_model_kf = grid_search_kf.best_estimator_
y_pred_kf = cross_val_predict(best_model_kf, X, y, cv=kf)

# Perform time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)
grid_search_ts = GridSearchCV(model, param_grid, cv=tscv, scoring=scoring, refit='neg_mean_absolute_error')
grid_search_ts.fit(X, y)

# Get the best model and predictions for TimeSeriesSplit
best_model_ts = grid_search_ts.best_estimator_
y_pred_ts = np.full_like(y, np.nan)
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train = y.iloc[train_index]
    best_model_ts.fit(X_train, y_train)
    y_pred_ts[test_index] = best_model_ts.predict(X_test)

# Extract true and predicted values
zn_true = df['Zn [ng/m**3]']
as_true = df['As [ng/m**3]']
zn_pred_kf = y_pred_kf[:, 0]
as_pred_kf = y_pred_kf[:, 1]

# Filter non-NaN predictions for TimeSeriesSplit
mask = ~np.isnan(y_pred_ts[:, 0])
zn_true_ts = zn_true[mask]
as_true_ts = as_true[mask]
zn_pred_ts = y_pred_ts[mask, 0]
as_pred_ts = y_pred_ts[mask, 1]

# Print results for k-fold cross-validation
print("Standard k-Fold Cross-Validation (k=10)")
print(f"Best parameters: {grid_search_kf.best_params_}")
print(f"Best score: {grid_search_kf.best_score_:.4f} (neg_mean_absolute_error)")
print("CV results for other metrics:")
for metric in scoring:
    mean_score = grid_search_kf.cv_results_[f'mean_test_{metric}'][grid_search_kf.best_index_]
    std_score = grid_search_kf.cv_results_[f'std_test_{metric}'][grid_search_kf.best_index_]
    print(f"{metric}: {mean_score:.4f} ± {std_score:.4f}")

print("\nEvaluation for Zn [ng/m³]:")
print(f"MAE: {mean_absolute_error(zn_true, zn_pred_kf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(zn_true, zn_pred_kf)):.4f}")
print(f"MSE: {mean_squared_error(zn_true, zn_pred_kf):.4f}")
print(f"R2: {r2_score(zn_true, zn_pred_kf):.4f}")
print(f"NSE: {nse(zn_true, zn_pred_kf):.4f}")
print(f"PCC: {pcc(zn_true, zn_pred_kf):.4f}")
print(f"MAPE: {mape(zn_true, zn_pred_kf):.4f}%")
print(f"PBIAS: {pbias(zn_true, zn_pred_kf):.4f}%")

print("\nEvaluation for As [ng/m³]:")
print(f"MAE: {mean_absolute_error(as_true, as_pred_kf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(as_true, as_pred_kf)):.4f}")
print(f"MSE: {mean_squared_error(as_true, as_pred_kf):.4f}")
print(f"R2: {r2_score(as_true, as_pred_kf):.4f}")
print(f"NSE: {nse(as_true, as_pred_kf):.4f}")
print(f"PCC: {pcc(as_true, as_pred_kf):.4f}")
print(f"MAPE: {mape(as_true, as_pred_kf):.4f}%")
print(f"PBIAS: {pbias(as_true, as_pred_kf):.4f}%")

# Print results for time series cross-validation
print("\nTime Series Cross-Validation (k=5)")
print(f"Best parameters: {grid_search_ts.best_params_}")
print(f"Best score: {grid_search_ts.best_score_:.4f} (neg_mean_absolute_error)")
print("CV results for other metrics:")
for metric in scoring:
    mean_score = grid_search_ts.cv_results_[f'mean_test_{metric}'][grid_search_ts.best_index_]
    std_score = grid_search_ts.cv_results_[f'std_test_{metric}'][grid_search_ts.best_index_]
    print(f"{metric}: {mean_score:.4f} ± {std_score:.4f}")

print("\nEvaluation for Zn [ng/m³] (TimeSeriesSplit, partial predictions):")
print(f"MAE: {mean_absolute_error(zn_true_ts, zn_pred_ts):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(zn_true_ts, zn_pred_ts)):.4f}")
print(f"MSE: {mean_squared_error(zn_true_ts, zn_pred_ts):.4f}")
print(f"R2: {r2_score(zn_true_ts, zn_pred_ts):.4f}")
print(f"NSE: {nse(zn_true_ts, zn_pred_ts):.4f}")
print(f"PCC: {pcc(zn_true_ts, zn_pred_ts):.4f}")
print(f"MAPE: {mape(zn_true_ts, zn_pred_ts):.4f}%")
print(f"PBIAS: {pbias(zn_true_ts, zn_pred_ts):.4f}%")

print("\nEvaluation for As [ng/m³] (TimeSeriesSplit, partial predictions):")
print(f"MAE: {mean_absolute_error(as_true_ts, as_pred_ts):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(as_true_ts, as_pred_ts)):.4f}")
print(f"MSE: {mean_squared_error(as_true_ts, as_pred_ts):.4f}")
print(f"R2: {r2_score(as_true_ts, as_pred_ts):.4f}")
print(f"NSE: {nse(as_true_ts, as_pred_ts):.4f}")
print(f"PCC: {pcc(as_true_ts, as_pred_ts):.4f}")
print(f"MAPE: {mape(as_true_ts, as_pred_ts):.4f}%")
print(f"PBIAS: {pbias(as_true_ts, as_pred_ts):.4f}%")
print(f"\nNote: TimeSeriesSplit predictions cover {len(zn_true_ts)}/{len(zn_true)} samples due to non-partitioning nature of splits.")

KeyboardInterrupt: 